In [1]:
import os
import requests
import pandas as pd
import json

from dotenv import load_dotenv
load_dotenv()

import pandas_datareader as web
import datetime as dt

%matplotlib inline

In [2]:
start = dt.datetime(2014,9,16)
end = dt.datetime.now()

btc_date = dt.datetime(2014,9,16)

iex_api_key = os.getenv("IEX_API_KEY")
iex_test_api_key = os.getenv("IEX_TEST_API_KEY")

base_url = 'https://cloud.iexapis.com/stable/'
sandbox_url = 'https://sandbox.iexapis.com/stable/'


token = os.environ.get('IEX_API_KEY')
test_token = os.environ.get('IEX_TEST_API_KEY')
params = {'token': token}
test_params = {'token': test_token}

test_resp = requests.get(base_url + 'status')
test_resp

<Response [200]>

In [3]:
def get_close(ticker):
    data = web.DataReader(ticker, 'yahoo', start, end)
    df = data[["Close"]]
    df.rename(columns={'Close':ticker},inplace=True)
    return df

In [4]:
def yahoo_10_year_data(tickers_list):
#     start = dt.datetime(2011,9,20)
#     end = dt.datetime.now()
    combined_df_f = pd.DataFrame()
    
    for i in tickers_list:
        if i == 0: 
            raw_df_1 = web.DataReader(i, 'yahoo', start, end)
            combined_df_f = raw_df_1[['Close']]
            combined_df_f.rename(columns={'Close':i},inplace=True)
            
        else:
            raw_df_2 = web.DataReader(i, 'yahoo', start, end)
            df_f = raw_df_2[['Close']]
            df_f.rename(columns={'Close':i},inplace=True)
            
            combined_df_f = pd.concat([combined_df_f,df_f], axis='columns')
            #combined_df_f = combined_df_f.merge(df_f, how='inner',right_index=True, left_index=True)


    return combined_df_f

def test_iex_apicall_get_close_10_years(tickers_list):
    combined_df_f = pd.DataFrame()
    
    for i in tickers_list: 
        
        if i == 0: 
            resp_data = requests.get(sandbox_url+'stock/'+i+'/chart/10y?token='+test_token)
            resp_df = pd.DataFrame(resp_data.json())
            
            resp_df.rename(columns={'date':'Date'},inplace=True)
            resp_df['Date'] = pd.to_datetime(resp_df['Date'])
            data = resp_df.set_index('Date')
            combined_df_f = data[['close']]
            symbol = i 
            combined_df_f.rename(columns={'close':symbol},inplace=True)
            
        else: 
            resp_data = requests.get(sandbox_url+'stock/'+i+'/chart/10y?token='+test_token)
            resp_df = pd.DataFrame(resp_data.json())
            
            resp_df.rename(columns={'date':'Date'},inplace=True)
            resp_df['Date'] = pd.to_datetime(resp_df['Date'])
            data = resp_df.set_index('Date')
            df_f = data[['close']]
            #symbol = data.iloc[0,4]
            symbol = i
            df_f.rename(columns={'close':symbol},inplace=True)
            
            combined_df_f = pd.concat([combined_df_f,df_f], axis='columns')
            
    return combined_df_f 

In [5]:
btc_test = get_close('BTC-USD')
btc_test

C:\Users\watso\anaconda3\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,BTC-USD
Date,
2014-09-16,457.334015
2014-09-17,424.440002
2014-09-18,394.795990
2014-09-19,408.903992
2014-09-20,398.821014
...,...
2021-09-20,40693.675781
2021-09-21,43574.507812
2021-09-22,44895.097656


In [6]:
btc_data_test = btc_test.loc['2014-09-20':]
btc_data_test
# data.iloc[0,4]

,BTC-USD
Date,
2014-09-20,398.821014
2014-09-21,402.152008
2014-09-22,435.790985
2014-09-23,423.204987
2014-09-24,411.574005
...,...
2021-09-20,40693.675781
2021-09-21,43574.507812
2021-09-22,44895.097656


In [7]:
gme_test = get_close('GME')
gme_test

C:\Users\watso\anaconda3\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,GME
Date,
2014-09-16,44.849998
2014-09-17,44.020000
2014-09-18,44.230000
2014-09-19,43.380001
2014-09-22,43.380001
...,...
2021-09-20,192.199997
2021-09-21,189.949997
2021-09-22,190.139999


In [8]:
combined_df_test = btc_test.merge(gme_test, how='inner',right_index=True, left_index=True)

In [9]:
combined_df_test

,BTC-USD,GME
Date,,
2014-09-16,457.334015,44.849998
2014-09-17,424.440002,44.020000
2014-09-18,394.795990,44.230000
2014-09-19,408.903992,43.380001
2014-09-22,435.790985,43.380001
...,...,...
2021-09-17,48278.363281,204.970001
2021-09-20,40693.675781,192.199997
2021-09-21,43574.507812,189.949997


In [10]:
stock_list = ['GME','AMC','AMZN','WMT','NVDA','AMD','AAPL','MSFT','DIS','PYPL']
stock_test = yahoo_10_year_data(stock_list)
stock_test

C:\Users\watso\anaconda3\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,GME,AMC,AMZN,WMT,NVDA,AMD,AAPL,MSFT,DIS,PYPL
Date,,,,,,,,,,
2014-09-16,44.849998,24.700001,327.760010,76.320000,4.785000,3.810000,25.215000,46.759998,90.269997,NaN
2014-09-17,44.020000,24.790001,324.000000,76.239998,4.787500,3.820000,25.395000,46.520000,90.330002,NaN
2014-09-18,44.230000,24.690001,325.000000,76.220001,4.860000,3.880000,25.447500,46.680000,90.339996,NaN
2014-09-19,43.380001,24.600000,331.320007,76.839996,4.770000,3.810000,25.240000,47.520000,90.489998,NaN
2014-09-22,43.380001,24.590000,324.500000,76.309998,4.722500,3.760000,25.264999,47.060001,89.290001,NaN
...,...,...,...,...,...,...,...,...,...,...
2021-09-20,192.199997,40.290001,3355.729980,142.740005,211.130005,101.550003,142.940002,294.299988,178.610001,269.910004
2021-09-21,189.949997,38.820000,3343.629883,143.020004,212.460007,102.820000,143.429993,294.799988,171.169998,269.489990
2021-09-22,190.139999,40.080002,3380.050049,142.690002,219.410004,104.379997,145.850006,298.579987,173.649994,272.339996


In [11]:
combined_df_test_2 = btc_test.merge(stock_test, how='inner',right_index=True, left_index=True)

In [12]:
combined_df_test_2

,BTC-USD,GME,AMC,AMZN,WMT,NVDA,AMD,AAPL,MSFT,DIS,PYPL
Date,,,,,,,,,,,
2014-09-16,457.334015,44.849998,24.700001,327.760010,76.320000,4.785000,3.810000,25.215000,46.759998,90.269997,NaN
2014-09-17,424.440002,44.020000,24.790001,324.000000,76.239998,4.787500,3.820000,25.395000,46.520000,90.330002,NaN
2014-09-18,394.795990,44.230000,24.690001,325.000000,76.220001,4.860000,3.880000,25.447500,46.680000,90.339996,NaN
2014-09-19,408.903992,43.380001,24.600000,331.320007,76.839996,4.770000,3.810000,25.240000,47.520000,90.489998,NaN
2014-09-22,435.790985,43.380001,24.590000,324.500000,76.309998,4.722500,3.760000,25.264999,47.060001,89.290001,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2021-09-17,48278.363281,204.970001,44.200001,3462.520020,144.729996,219.000000,103.879997,146.059998,299.869995,183.470001,276.339996
2021-09-20,40693.675781,192.199997,40.290001,3355.729980,142.740005,211.130005,101.550003,142.940002,294.299988,178.610001,269.910004
2021-09-21,43574.507812,189.949997,38.820000,3343.629883,143.020004,212.460007,102.820000,143.429993,294.799988,171.169998,269.489990


In [13]:
iex_stock_list = test_iex_apicall_get_close_10_years(stock_list)
iex_stock_list

C:\Users\watso\anaconda3\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,GME,AMC,AMZN,WMT,NVDA,AMD,AAPL,MSFT,DIS,PYPL
Date,,,,,,,,,,
2011-09-26,24.15,NaN,238.57,52.50,3.5790,6.40,14.8901,26.57,30.80,NaN
2011-09-27,24.22,NaN,230.32,53.27,3.6000,6.66,14.7328,26.88,32.38,NaN
2011-09-28,24.28,NaN,230.13,53.25,3.3910,6.41,14.7496,25.62,31.78,NaN
2011-09-29,24.55,NaN,228.81,52.28,3.4035,5.31,14.1196,26.60,31.87,NaN
2011-09-30,23.10,NaN,226.52,54.00,3.1742,5.25,14.1984,25.25,31.51,NaN
...,...,...,...,...,...,...,...,...,...,...
2021-09-20,192.20,41.99,3458.69,147.88,220.7500,101.80,146.9900,309.00,182.38,274.06
2021-09-21,194.93,39.11,3375.34,149.89,218.9900,105.60,149.6100,299.70,172.66,271.28
2021-09-22,197.63,41.75,3502.33,143.86,228.3600,108.53,151.4500,305.57,181.15,276.51


In [14]:
combined_df_test_3 = btc_test.merge(iex_stock_list, how='inner',right_index=True, left_index=True)
combined_df_test_3

,BTC-USD,GME,AMC,AMZN,WMT,NVDA,AMD,AAPL,MSFT,DIS,PYPL
Date,,,,,,,,,,,
2014-09-16,457.334015,45.05,24.90,337.19,77.45,4.7854,3.83,25.9370,48.22,93.50,NaN
2014-09-17,424.440002,45.89,24.93,340.00,76.97,4.9323,3.99,25.5995,47.85,90.57,NaN
2014-09-18,394.795990,45.55,24.96,341.00,78.20,5.0400,4.05,26.2571,48.53,91.23,NaN
2014-09-19,408.903992,44.22,25.30,342.81,78.44,4.9700,3.98,26.0900,49.41,92.54,NaN
2014-09-22,435.790985,45.47,25.25,334.60,79.29,4.7368,3.83,25.3520,47.41,90.66,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2021-09-17,48278.363281,214.91,44.90,3491.91,147.04,224.0000,105.36,148.5800,301.66,188.55,287.79
2021-09-20,40693.675781,192.20,41.99,3458.69,147.88,220.7500,101.80,146.9900,309.00,182.38,274.06
2021-09-21,43574.507812,194.93,39.11,3375.34,149.89,218.9900,105.60,149.6100,299.70,172.66,271.28


In [15]:
eth_test = get_close('ETH-USD')
eth_test

C:\Users\watso\anaconda3\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,ETH-USD
Date,
2015-08-06,2.772120
2015-08-07,0.753325
2015-08-08,0.701897
2015-08-09,0.708448
2015-08-10,1.067860
...,...
2021-09-20,2764.431152
2021-09-21,3077.867920
2021-09-22,3155.523682


In [16]:
crypto_df = btc_test.merge(eth_test, how='inner',right_index=True, left_index=True)
crypto_df

,BTC-USD,ETH-USD
Date,,
2015-08-06,279.584991,2.772120
2015-08-07,260.997009,0.753325
2015-08-08,265.083008,0.701897
2015-08-09,264.470001,0.708448
2015-08-10,270.385986,1.067860
...,...,...
2021-09-20,40693.675781,2764.431152
2021-09-21,43574.507812,3077.867920
2021-09-22,44895.097656,3155.523682


In [17]:
full_df = crypto_df.merge(iex_stock_list, how='inner',right_index=True, left_index=True)
full_df

,BTC-USD,ETH-USD,GME,AMC,AMZN,WMT,NVDA,AMD,AAPL,MSFT,DIS,PYPL
Date,,,,,,,,,,,,
2015-08-06,279.584991,2.772120,45.72,30.44,547.94,74.15,5.2895,2.13,30.0612,48.20,111.93,40.18
2015-08-07,260.997009,0.753325,46.08,30.25,536.96,73.51,5.9740,2.12,29.3100,48.20,114.31,40.31
2015-08-10,270.385986,1.067860,45.89,31.20,531.00,72.84,5.9610,1.97,30.8500,48.55,116.00,40.78
2015-08-11,266.376007,1.217440,47.36,29.04,545.78,72.65,6.0740,1.90,28.3896,48.42,109.00,38.98
2015-08-12,264.079987,1.827670,46.06,29.52,528.17,75.16,6.0300,2.00,29.8900,48.64,111.00,39.25
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-17,48278.363281,3432.018311,214.91,44.90,3491.91,147.04,224.0000,105.36,148.5800,301.66,188.55,287.79
2021-09-20,40693.675781,2764.431152,192.20,41.99,3458.69,147.88,220.7500,101.80,146.9900,309.00,182.38,274.06
2021-09-21,43574.507812,3077.867920,194.93,39.11,3375.34,149.89,218.9900,105.60,149.6100,299.70,172.66,271.28
